# Reading Data Homework

In [1]:
import numpy as np
from astropy.table import QTable
from astroquery.gaia import Gaia

ModuleNotFoundError: No module named 'astroquery'

### Get a list of ALL the Gaia objects you would see straight above your head (Zenith) at midnight tonight (10/12).

### You will need information from your [Astro Info Sheet](./Info/AstroInfo.pdf).

### At midnight tonight:

  - The Local Sidereal Time (LST) for Seattle (this room) at local midnight on Oct 12th is 1.23 deg.
  - Seattle (this room) has a latitude of +47.65 deg.

### You want all objects with 

- `ra` and `dec` within 1 degree of the Zenith
- `phot_g_mean_mag` < 10
- `parallax` > 0
- You also want the `source_id` and color index (`bp_rp`) for all objects

In [2]:
queryOne = """
SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, parallax, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE CONTAINS(
   POINT('ICRS', 1.23, 47.65),
   CIRCLE('ICRS', ra, dec, 1)
   ) = 1
AND phot_g_mean_mag < 10.0
AND parallax > 0
AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC
"""

In [3]:
print(queryOne)


SELECT TOP 100
source_id, ra, dec, phot_g_mean_mag, parallax, bp_rp
FROM gaiadr3.gaia_source_lite
WHERE CONTAINS(
   POINT('ICRS', 1.23, 47.65),
   CIRCLE('ICRS', ra, dec, 1)
   ) = 1
AND phot_g_mean_mag < 10.0
AND parallax > 0
AND bp_rp IS NOT NULL
ORDER BY bp_rp ASC



In [4]:
midnightJob = Gaia.launch_job(queryOne)

In [5]:
midnightTable = midnightJob.get_results()
midnightTable

source_id,ra,dec,phot_g_mean_mag,parallax,bp_rp
,deg,deg,mag,mas,mag
int64,float64,float64,float32,float64,float32
387265694864872064,0.34830775880952414,47.398038007347004,9.531222,1.1806670135937007,0.07384682
393289609837293312,0.9241862552473424,48.43686655391979,6.7911806,7.549027097652189,0.07444668
387205702762203392,1.4478364083048825,47.08728242495348,8.4606495,2.347040040356511,0.07517624
387248819938407424,0.7141248779233184,47.2522510421125,8.898067,3.8783236736864075,0.112083435
387248824234876544,0.7147348039925946,47.25206300931765,9.11861,3.9408446041799254,0.2881441
387281053668103936,0.5495253811134138,47.57717039958105,8.387774,3.0242721414173617,0.37495232
387262877366317184,0.09693714171567966,47.373832963263176,9.346837,1.9767377943486057,0.39721203
393300776752373760,0.7085133628612212,48.078576078927,8.905772,5.221212848471736,0.46898556


In [6]:
midnightTable.sort('phot_g_mean_mag')

## What is the `source_id` and `phot_g_mean_mag` of the brightest object?

In [7]:
brightStarSource = midnightTable['source_id'][0]
brightStarMag = midnightTable['phot_g_mean_mag'][0]
print(f'The brightest star at the zenith on Oct 12th 2022 has a source id of {brightStarSource} and a magnitude of {brightStarMag}')

The brightest star at the zenith on Oct 12th 2022 has a source id of 393289609837293312 and a magnitude of 6.791180610656738


## Add a column `distance` to your table that has the distance (in parsecs) for all of the objects

In [10]:
def distanceFinder(parallax):
    result = 1 / (parallax / 1000)
    return result

In [12]:
distanceArray = distanceFinder(midnightTable['parallax'])
midnightTable.add_column(distanceArray, name='Distance',index=5)
midnightTable

source_id,ra,dec,phot_g_mean_mag,parallax,Distance,bp_rp
,deg,deg,mag,mas,,mag
int64,float64,float64,float32,float64,float64,float32
393289609837293312,0.9241862552473424,48.43686655391979,6.7911806,7.549027097652189,132.46740103913635,0.07444668
387252260207420800,0.6258579059648612,47.42276308972577,7.507746,8.838562582470374,113.14056903136172,0.6523862
386987685925706624,1.795875822269915,46.75689909330566,7.634305,3.7110584958585857,269.4648982536831,1.2059174
393232125995342848,1.5789685405622853,47.72727464552025,7.981288,2.2976200860598652,435.232963912183,1.7836871
393368946475346304,1.3244609852984932,48.47712236815464,8.050094,2.9077545237532427,343.90798529623805,1.1151123
393033423626925824,1.5989694453744596,47.4047409892245,8.275679,0.8779702295231859,1138.990783939322,2.205091
392996452548763264,2.527986257042034,47.30682366556531,8.284404,0.6899484623077705,1449.3836201259949,1.29217
387263083524736000,0.059659651457552816,47.42418447234008,8.357033,0.7588008845935008,1317.868785215917,3.3809414


## What is the `source_id` and `distance` of the nearest object?

In [15]:
midnightTable.sort('Distance')
closest = midnightTable['Distance'][0]
sourceClose = midnightTable['source_id'][0]
print(f'The closest star has a distance of {closest}pc and a source id of {sourceClose}')

The closest star has a distance of 77.05780874697739pc and a source id of 387170209152367360


## What is the `source_id`, `distance` and color index of the bluest object?

In [16]:
midnightTable.sort('bp_rp')
blueClose = midnightTable['Distance'][0]
blueSource = midnightTable['source_id'][0]
print(f'The bluest star has an id {blueSource} and at a distance of {blueClose}pc')

The bluest star has an id 387265694864872064 and at a distance of 846.9788589724477pc


## What is the `source_id`, `distance` and color index of the reddest object?

In [17]:
midnightTable.sort('bp_rp', reverse=True)
redClose = midnightTable['Distance'][0]
redSource = midnightTable['source_id'][0]
print(f'The reddest star has an id {redSource} and at a distance of {redClose}pc')

The reddest star has an id 387263083524736000 and at a distance of 1317.868785215917pc


### Due Mon Oct 17 - 1 pm
- `Save notebook as .html: File -> Download as -> HTML`
- `Upload .html to Canvas`